## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [ ]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")

### 版本与Pydantic兼容
在pytho SDK中，目前langchain已经发布了0.0/0.1/0.2/0.3四个大的版本，其中各个版本的Pydantic兼容性如下：
- langchain<0.0.267版本，仅支持pydantic v1
- langchain>0.0.267版本，支持pydantic v1和v2共存，在langchain内部空间使用Pydantic2的v1命名空间，继续使用v1
- langchain 0.3.0 仅支持pydantic v2
- 注意：大部分的版本兼容问题可能都是由pydantic引起的
- 虽然 LangChain 在某些 API 中支持 Pydantic V2 对象（如下所列），但建议用户继续使用 Pydantic V1 对象，直到 LangChain 0.3 发布。
****

### 大多数用于工具使用的 LangChain API都已更新为接受 Pydantic v1 或 v2 对象。

- BaseChatModel.bind_tools【pydantic v1支持/pydantic v2需要langchain-core>=0.2.23&适合合作伙伴软件包的版本】
- BaseChatModel.with_structured_output【pydantic v1支持/pydantic v2需要langchain-core>=0.2.23&适合合作伙伴软件包的版本】
- Tool.from_function【pydantic v1支持/pydantic v2需要langchain-core>=0.2.23&适合合作伙伴软件包的版本】
- StructuredTool.from_function【pydantic v1支持/pydantic v2需要langchain-core>=0.2.23&适合合作伙伴软件包的版本】
****

### 通过 bind_tools 或 with_structured_output API 接受 pydantic v2 对象的合作伙伴包

- langchain-mistralai 【pydantic v1支持/v2需自身版本>=0.1.11】
- langchain-anthropic 【pydantic v1支持/v2需自身版本>=0.1.21】
- langchain-robocorp 【pydantic v1支持/v2需自身版本>=0.0.10】
- langchain-openai 【pydantic v1支持/v2需自身版本>=0.1.19】
- langchain-fireworks 【pydantic v1支持/v2需自身版本>=0.1.5】
- langchain-aws 【pydantic v1支持/v2需自身版本>=0.1.15】
*****

### 在langchain-core<0.2.23版本之前你应该：

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel # <-- 注意 v1 namespace

class Person(BaseModel):
    """Personal information"""
    name: str
    
model = ChatOpenAI()
model = model.with_structured_output(Person)

model.invoke('Bob is a person.')

### 在langchain-core>=0.2.23之后你应该：

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

class Person(BaseModel):
    """Personal information"""
    name: str
    
    
model = ChatOpenAI()
model = model.with_structured_output(Person)

model.invoke('Bob is a person.')

### 如果你要继承langchain的原型 你应该注意使用v1命名空间：

In [ ]:
from pydantic.v1 import validator
from langchain_core.tools import BaseTool

class CustomTool(BaseTool): # BaseTool is v1 code
    x: int = Field(default=1)

    def _run(*args, **kwargs):
        return "hello"

    @validator('x') # v1 code
    @classmethod
    def validate_x(cls, x: int) -> int:
        return 1
    

CustomTool(
    name='custom_tool',
    description="hello",
    x=1,
)

### pydantic v1 和v2混用会产生神秘错误

In [ ]:
# bad case
from pydantic import Field, field_validator # pydantic v2
from langchain_core.tools import BaseTool

class CustomTool(BaseTool): # BaseTool is v1 code
    x: int = Field(default=1)

    def _run(*args, **kwargs):
        return "hello"

    @field_validator('x') # v2 code
    @classmethod
    def validate_x(cls, x: int) -> int:
        return 1
    

CustomTool( 
    name='custom_tool',
    description="hello",
    x=1,
)

### 合作包中使用pydantic v1的会后，可以开启跳过验证

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI # <-- ChatOpenAI uses pydantic v1
from pydantic import BaseModel, SkipValidation


class Foo(BaseModel): # <-- BaseModel is from Pydantic v2
    model: Annotated[ChatOpenAI, SkipValidation()]

Foo(model=ChatOpenAI(api_key="hello"))